<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DSE-203:-Final-Project" data-toc-modified-id="DSE-203:-Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DSE 203: Final Project</a></span><ul class="toc-item"><li><span><a href="#Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag" data-toc-modified-id="Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Author: Adelle Driker, Bo Yan, Paul Cabasag</a></span></li><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Yelp:Tips-to-FoodOn-ValueMatching" data-toc-modified-id="Yelp:Tips-to-FoodOn-ValueMatching-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Yelp:Tips to FoodOn ValueMatching</a></span><ul class="toc-item"><li><span><a href="#Get-Business_Id-and-Text-From-Yelp:Tips-Nodes-&amp;-Format-as-Dataframe" data-toc-modified-id="Get-Business_Id-and-Text-From-Yelp:Tips-Nodes-&amp;-Format-as-Dataframe-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Get Business_Id and Text From Yelp:Tips Nodes &amp; Format as Dataframe</a></span></li><li><span><a href="#Extract-Nouns-from-Each-Tip-Using-NLTK" data-toc-modified-id="Extract-Nouns-from-Each-Tip-Using-NLTK-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Extract Nouns from Each Tip Using NLTK</a></span></li><li><span><a href="#Read-in-Ingredients-File" data-toc-modified-id="Read-in-Ingredients-File-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Read in Ingredients File</a></span></li><li><span><a href="#Create-a-List-of-Matches-for-Each-Business_Id,-Tip-Pair" data-toc-modified-id="Create-a-List-of-Matches-for-Each-Business_Id,-Tip-Pair-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Create a List of Matches for Each Business_Id, Tip Pair</a></span></li><li><span><a href="#Create-a-Dictionary-to-Store-all-Matched-FoodOn-Items-for-each-Business_Id" data-toc-modified-id="Create-a-Dictionary-to-Store-all-Matched-FoodOn-Items-for-each-Business_Id-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Create a Dictionary to Store all Matched FoodOn Items for each Business_Id</a></span></li><li><span><a href="#Remove-any-Business_Id-Keys-Without-a-Match-List" data-toc-modified-id="Remove-any-Business_Id-Keys-Without-a-Match-List-1.3.6"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span>Remove any Business_Id Keys Without a Match List</a></span></li><li><span><a href="#Create-a-NodeMatcher-Object-and-Set-a-Default-Value-for-the-New-Ingredient_Matches-Property" data-toc-modified-id="Create-a-NodeMatcher-Object-and-Set-a-Default-Value-for-the-New-Ingredient_Matches-Property-1.3.7"><span class="toc-item-num">1.3.7&nbsp;&nbsp;</span>Create a NodeMatcher Object and Set a Default Value for the New Ingredient_Matches Property</a></span></li><li><span><a href="#Set-the-Ingredient_Matches-Property-and-Push-to-Remote-Graph" data-toc-modified-id="Set-the-Ingredient_Matches-Property-and-Push-to-Remote-Graph-1.3.8"><span class="toc-item-num">1.3.8&nbsp;&nbsp;</span>Set the Ingredient_Matches Property and Push to Remote Graph</a></span></li></ul></li></ul></li></ul></div>

# DSE 203: Final Project
## Author: Adelle Driker, Bo Yan, Paul Cabasag

## Import Libraries

In [7]:
import pandas as pd
import numpy as np
import nltk
from py2neo import Graph, Node, Relationship, NodeMatcher
from collections import Counter

## Yelp:Tips to FoodOn ValueMatching

In [8]:
#Connect to Neo4j Database
url="bolt://localhost:7687"
pwd="12345"
graph=Graph(url, auth=('neo4j', pwd))

### Get Business_Id and Text From Yelp:Tips Nodes & Format as Dataframe

In [9]:
query = """
        MATCH (n:Photos)-[:PHOTOS_TO_BUSINESS]-(m:Business) WHERE m.categories CONTAINS 'Food' 
        AND size(n.caption) > 10 RETURN n.business_id, n.caption ORDER BY n.business_id
"""
df_photos = graph.query(query).to_data_frame()
df_photos

,n.business_id,n.caption
0,--164t1nclzzmca7eDiJMw,Standard angled shot of the trailer
1,--164t1nclzzmca7eDiJMw,Red Curry Noodles w/ BBQ Pork ($8.50) size large
2,--hkbIWgBKBOZq4VcNwdhQ,Large side lot becomes a patio extension durin...
3,--hkbIWgBKBOZq4VcNwdhQ,Flights are a good way to go. My favorite is t...
4,--hkbIWgBKBOZq4VcNwdhQ,Flight time
...,...,...
40800,zz4JTDTYIuY-TVdQ5zViSw,Proper spacing
40801,zz9reWVOhZXVgZcA6aNyDg,Fresh fruit
40802,zzlkjDG9Rv8Jn-vSolMgyw,Grits and fresh fruit
40803,zzlkjDG9Rv8Jn-vSolMgyw,Bread Pudding


### Extract Nouns from Each Tip Using NLTK

Since each food item is expected to be a noun, we extract only nouns from each tip.

In [10]:
%%time

### Extract nouns from text
is_noun = lambda pos: pos[:2] == 'NN'

nouns = []
stop_words = ['get', 'mi', 'place', 'time', 'you', 'me', 'location', 'owner', 'vibe', 'service', 'i', 'Sunday', 'Monday', 
              'Tuesday', 'Wednesday','Thursday', 'Friday', 'Saturday', 'lots', 'sale', 'awesome', 'bahn', 'we', 'us', 'them']

df_photos['n.caption'] = df_photos['n.caption'].str.lower()

for i in range(df_photos.shape[0]):
    tokenized = nltk.word_tokenize(df_photos['n.caption'][i])
    nouns_text = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos) and word not in stop_words]
    nouns.append(nouns_text)

Wall time: 1min 4s


### Read in Ingredients File

In [11]:
ingr = pd.read_csv('ingredients.csv')

### Create a List of Matches for Each Business_Id, Tip Pair

In [12]:
%%time

matches = []

for i in range(len(nouns)):
    
    w_matches = []
    
    for j in range(ingr.shape[0]):

        if ingr['Ingredients'][j] in nouns[i]:
            w_matches.append([ingr['Ingredients'][j]])
    matches.append(w_matches)

# Add 'matches' column to df_tips
df_photos['matches'] = matches

Wall time: 1min 1s


### Create a Dictionary to Store all Matched FoodOn Items for each Business_Id

In [13]:
%%time
match_dict = {}

for i in range(df_photos.shape[0]):
    curr_id = df_photos['n.business_id'][i]
    if curr_id in match_dict:
        match_dict[curr_id].append(df_photos['matches'][i])
    else:
        match_dict[curr_id] = df_photos['matches'][i]
        

for key in match_dict:
    match_dict[key] = list(np.unique([i if type(item) == list else item for sublist in match_dict[key] for item in sublist for i in item]))

#match_dict


Wall time: 1.14 s


### Remove any Business_Id Keys Without a Match List

In [14]:
for key in list(match_dict.keys()):
    if len(match_dict[key]) == 0:
        del match_dict[key]
        
#match_dict

### Create a NodeMatcher Object and Set a Default Value for the New Ingredient_Matches Property

The ingredient_matches property will store the list of matched ingredients, which will be used to create edges between a Yelp Business an Ingredient item.

In [15]:
matcher = NodeMatcher(graph)

In [16]:
%%time
for key in match_dict:
    curr_node = matcher.match("Business", business_id = key).first()
    curr_node.setdefault("photos_ingredient_matches", "")
    graph.push(curr_node)

Wall time: 17min 24s


### Set the Ingredient_Matches Property and Push to Remote Graph

In [17]:
%%time
for key in match_dict:
    graph.run("MATCH (b:Business {business_id: '%s'}) SET b.photos_ingredient_matches = $pim" % str(key), pim = str(match_dict[key]))

Wall time: 24min 15s


In [18]:
%%time
for key in match_dict:
    curr_node = matcher.match("Business", business_id = key).first()
    #print(curr_node)
    graph.push(curr_node)

Wall time: 28min 4s
